# 🧪 DataHub Glossary를 온톨로지로 쓸 수 있을까 — 실습

> **DataNexus Building Log #003** 실습 노트북
>
> 📝 블로그 원문: [DataHub Glossary를 온톨로지로 쓸 수 있을까](https://biz-agentic-ai.github.io/posts/datanexus/003-datahub-glossary-as-ontology/)

---
## 이 노트북의 목표

블로그 글의 핵심 내용을 **직접 코드로 따라하며** 이해합니다.

| 섹션 | 내용 | 핵심 질문 |
|:---:|------|----------|
| 1 | 온톨로지 30초 정리 | 그래프가 뭔데? 노드, 엣지가 뭔데? |
| 2 | 문제 재현 | `RelatedTo` 하나로 퉁치면 왜 안 되나? |
| 3 | 해결 | 관계를 나누면 뭐가 달라지나? |
| 4 | 엣지 속성 | 관계에 정보를 붙이면? |
| 5 | 파생 지표 | 계산식도 그래프로 표현할 수 있다고? |
| 6 | 전체 시각화 | 한눈에 보기 |
| 7 | 정리 | 되는 것 vs 안 되는 것 |

> ⚠️ **DozerDB/Neo4j 없이도 실행 가능합니다.** Python의 NetworkX 라이브러리로 그래프 DB 개념을 시뮬레이션합니다.

## 0. 환경 설정

In [ ]:
# Colab 기본 라이브러리로 충분합니다
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.font_manager import FontProperties
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Colab 환경)
import os
if not os.path.exists('/usr/share/fonts/truetype/nanum/NanumGothic.ttf'):
    !apt-get -qq install -y fonts-nanum > /dev/null 2>&1

plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

# 이후 셀에서 사용할 폰트 상수
FONT_NAME = 'NanumGothic'

print('✅ 환경 설정 완료!')

---
## 1. 온톨로지란? — 30초 개념 정리

어렵게 생각할 필요 없습니다. 핵심은 딱 두 가지입니다.

### 그래프 = 노드(동그라미) + 엣지(화살표)

- **노드(Node)**: 개념이나 용어를 나타냅니다. 예) "매출", "공장", "제품"
- **엣지(Edge)**: 노드 사이의 **관계**를 나타냅니다. 예) "순매출 →[IsA]→ 매출"

### 온톨로지 = 업무 지식을 그래프로 표현한 것

사람 머릿속에 있는 지식을 기계가 읽을 수 있게 정리한 것이 온톨로지입니다.

> 💡 **"순매출은 매출의 한 종류다"** 를 그래프로 표현하면?
> - 노드 2개: `순매출`, `매출`
> - 엣지 1개: `순매출 →[IsA]→ 매출`

아래 코드로 직접 만들어 봅시다.

In [ ]:
# 📌 가장 간단한 온톨로지: 노드 2개, 엣지 1개
G = nx.DiGraph()

# 노드 추가 (= 용어 등록)
G.add_node('순매출', definition='총매출에서 반품과 에누리를 뺀 금액')
G.add_node('매출',   definition='사업 활동으로 발생하는 수익')

# 엣지 추가 (= 관계 설정)
G.add_edge('순매출', '매출', relation='IsA')

# 시각화
fig, ax = plt.subplots(figsize=(6, 3))
pos = {'순매출': (0, 0), '매출': (2, 0)}

nx.draw_networkx_nodes(G, pos, ax=ax, node_color='#3498db', node_size=2000, alpha=0.9)
nx.draw_networkx_labels(G, pos, ax=ax, font_family=FONT_NAME, font_size=14, font_color='white', font_weight='bold')
nx.draw_networkx_edges(G, pos, ax=ax, edge_color='#2c3e50', arrows=True, arrowsize=20, width=2)
nx.draw_networkx_edge_labels(G, pos, ax=ax,
    edge_labels={('순매출', '매출'): 'IsA'},
    font_family=FONT_NAME, font_size=12, font_color='#e74c3c')

ax.set_title('가장 간단한 온톨로지: "순매출은 매출의 한 종류"', fontsize=14, fontfamily=FONT_NAME)
ax.axis('off')
plt.tight_layout()
plt.show()

print()
print('💡 이게 온톨로지의 전부입니다!')
print('   - 동그라미(노드) = 용어')
print('   - 화살표(엣지) = 관계')
print('   - 실제로는 노드와 엣지가 수십~수백 개로 확장됩니다.')

---
## 2. 문제 재현: `RelatedTo` 하나로 퉁치면 생기는 일

> 🎯 **이 섹션에서 확인할 것**
> - DataHub Glossary의 관계 유형은 4개뿐 (IsA, HasA, RelatedTo, Values)
> - "생산"과 "재고" 같은 서로 다른 의미의 관계가 모두 `RelatedTo`로 표현됨
> - 그 결과, NL2SQL 엔진이 잘못된 테이블을 JOIN할 수 있음

블로그에서 다뤘던 핵심 문제입니다:
> "A 공장에서 **생산된** B 제품"과 "A 공장에 **재고가 있는** B 제품".
> DataHub Glossary에서는 둘 다 `RelatedTo`.

In [ ]:
# --- DataHub 방식: RelatedTo 하나로 표현 ---
datahub_graph = nx.MultiDiGraph()

datahub_graph.add_node('A공장', type='Factory', definition='서울 소재 공장')
datahub_graph.add_node('B제품', type='Product', definition='주력 제품')

# DataHub Glossary에서는 관계 유형이 4개뿐
# 생산과 재고 모두 RelatedTo로 표현할 수밖에 없다
datahub_graph.add_edge('A공장', 'B제품', relation='RelatedTo')  # 생산? 재고?
datahub_graph.add_edge('A공장', 'B제품', relation='RelatedTo')  # 생산? 재고?

print('=== DataHub Glossary 방식 ===')
print(f'노드: {list(datahub_graph.nodes(data=True))}')
print(f'엣지: {list(datahub_graph.edges(data=True))}')
print()
print('👆 엣지가 2개인데, 둘 다 "RelatedTo"입니다.')
print('   어느 것이 "생산"이고 어느 것이 "재고"인지 구분이 안 됩니다!')

In [ ]:
# 📊 시각화: DataHub 방식 (Before)
fig, ax = plt.subplots(figsize=(7, 4))
pos = {'A공장': (0, 0), 'B제품': (2, 0)}

colors = {'Factory': '#e74c3c', 'Product': '#3498db'}
node_colors = [colors[datahub_graph.nodes[n]['type']] for n in datahub_graph.nodes()]
nx.draw_networkx_nodes(datahub_graph, pos, ax=ax, node_color=node_colors, node_size=2500, alpha=0.9)
nx.draw_networkx_labels(datahub_graph, pos, ax=ax, font_family=FONT_NAME, font_size=13, font_color='white', font_weight='bold')

nx.draw_networkx_edges(datahub_graph, pos, ax=ax, edge_color='#95a5a6',
                       arrows=True, arrowsize=20, width=3, connectionstyle='arc3,rad=0.1')
nx.draw_networkx_edges(datahub_graph, pos, ax=ax, edge_color='#95a5a6',
                       arrows=True, arrowsize=20, width=3, connectionstyle='arc3,rad=-0.1')

ax.text(1, 0.25, 'RelatedTo (생산?)', ha='center', fontsize=11, color='#95a5a6', fontfamily=FONT_NAME)
ax.text(1, -0.25, 'RelatedTo (재고?)', ha='center', fontsize=11, color='#95a5a6', fontfamily=FONT_NAME)
ax.set_title('❌ DataHub 방식: 관계가 구분되지 않음', fontsize=14, fontfamily=FONT_NAME, color='#e74c3c')
ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# 💬 시뮬레이션: "A 공장에서 생산된 제품은?" 질문 처리
print('💬 사용자 질문: "A 공장에서 생산된 제품은?"')
print()

edges = list(datahub_graph.edges(data=True))
related = [e for e in edges if e[0] == 'A공장' and e[2]['relation'] == 'RelatedTo']
print(f'🔍 온톨로지 조회: A공장 → RelatedTo → {[e[1] for e in related]}')
print(f'   관계 {len(related)}개 발견, 모두 "RelatedTo"')
print()
print('   ❌ 어느 것이 "생산"이고 어느 것이 "재고"인지 구분 불가!')
print('   ❌ LLM이 production 대신 inventory 테이블을 JOIN할 수 있음')

---
## 3. 해결: DozerDB 방식 — 세분화된 관계 유형

> 🎯 **이 섹션에서 확인할 것**
> - 관계 유형을 `MANUFACTURES`, `STOCKS` 등 의미별로 분리
> - 같은 질문에 대해 정확한 답을 내는 과정
> - 블로그의 Cypher ↔ Python 대응 관계

```
Cypher (DozerDB):    CREATE (factory)-[:MANUFACTURES {...}]->(product)
Python (NetworkX):   G.add_edge('A공장', 'B제품', key='MANUFACTURES', ...)
```

In [ ]:
# --- DozerDB 방식: 세분화된 관계 유형 + 엣지 속성 ---
dozerdb_graph = nx.MultiDiGraph()

dozerdb_graph.add_node('A공장', type='Factory', definition='서울 소재 공장')
dozerdb_graph.add_node('B제품', type='Product', definition='주력 제품')

dozerdb_graph.add_edge('A공장', 'B제품', key='MANUFACTURES', since='2024-01-01', confidence=0.95)
dozerdb_graph.add_edge('A공장', 'B제품', key='STOCKS', quantity=500, last_updated='2026-02-01')

print('=== DozerDB 방식 ===')
for u, v, key, data in dozerdb_graph.edges(keys=True, data=True):
    print(f'  {u} —[{key}]→ {v}')
    for k, val in data.items():
        print(f'      {k}: {val}')
    print()
print('👆 같은 두 노드 사이에 의미가 다른 관계가 명확히 구분됩니다!')

In [ ]:
# 📊 Before vs After 비교 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
pos = {'A공장': (0, 0), 'B제품': (2, 0)}
colors = {'Factory': '#e74c3c', 'Product': '#3498db'}

# ===== Before: DataHub =====
nc1 = [colors[datahub_graph.nodes[n]['type']] for n in datahub_graph.nodes()]
nx.draw_networkx_nodes(datahub_graph, pos, ax=ax1, node_color=nc1, node_size=2500, alpha=0.9)
nx.draw_networkx_labels(datahub_graph, pos, ax=ax1, font_family=FONT_NAME, font_size=13, font_color='white', font_weight='bold')
nx.draw_networkx_edges(datahub_graph, pos, ax=ax1, edge_color='#95a5a6', arrows=True, arrowsize=20, width=3, connectionstyle='arc3,rad=0.1')
nx.draw_networkx_edges(datahub_graph, pos, ax=ax1, edge_color='#95a5a6', arrows=True, arrowsize=20, width=3, connectionstyle='arc3,rad=-0.1')
ax1.text(1, 0.25, 'RelatedTo (?)', ha='center', fontsize=11, color='#95a5a6', fontfamily=FONT_NAME)
ax1.text(1, -0.25, 'RelatedTo (?)', ha='center', fontsize=11, color='#95a5a6', fontfamily=FONT_NAME)
ax1.set_title('❌ Before: DataHub', fontsize=14, fontfamily=FONT_NAME, color='#e74c3c')
ax1.axis('off')

# ===== After: DozerDB =====
nc2 = [colors[dozerdb_graph.nodes[n]['type']] for n in dozerdb_graph.nodes()]
nx.draw_networkx_nodes(dozerdb_graph, pos, ax=ax2, node_color=nc2, node_size=2500, alpha=0.9)
nx.draw_networkx_labels(dozerdb_graph, pos, ax=ax2, font_family=FONT_NAME, font_size=13, font_color='white', font_weight='bold')
nx.draw_networkx_edges(dozerdb_graph, pos, ax=ax2, edge_color='#27ae60', arrows=True, arrowsize=20, width=3, connectionstyle='arc3,rad=0.1')
nx.draw_networkx_edges(dozerdb_graph, pos, ax=ax2, edge_color='#f39c12', arrows=True, arrowsize=20, width=3, connectionstyle='arc3,rad=-0.1')
ax2.text(1, 0.30, 'MANUFACTURES ✅', ha='center', fontsize=11, color='#27ae60', fontfamily=FONT_NAME, fontweight='bold')
ax2.text(1, -0.30, 'STOCKS ✅', ha='center', fontsize=11, color='#f39c12', fontfamily=FONT_NAME, fontweight='bold')
ax2.set_title('✅ After: DozerDB', fontsize=14, fontfamily=FONT_NAME, color='#27ae60')
ax2.axis('off')

plt.suptitle('관계 유형 분리의 효과', fontsize=16, fontfamily=FONT_NAME, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 💬 같은 질문, 다른 결과!
print('💬 사용자 질문: "A 공장에서 생산된 제품은?"')
print()

manufactures = [(u, v, data) for u, v, key, data in dozerdb_graph.edges(keys=True, data=True)
                if u == 'A공장' and key == 'MANUFACTURES']
stocks = [(u, v, data) for u, v, key, data in dozerdb_graph.edges(keys=True, data=True)
          if u == 'A공장' and key == 'STOCKS']

print(f'🔍 온톨로지 조회: A공장 → MANUFACTURES → {[e[1] for e in manufactures]}')
print(f'   confidence: {manufactures[0][2]["confidence"]}')
print(f'   ✅ production 테이블 JOIN → 정확한 결과!')
print()
print(f'📦 참고: A공장 → STOCKS → {[e[1] for e in stocks]}')
print(f'   quantity: {stocks[0][2]["quantity"]}')
print(f'   → 이 질문과 무관 → 자동 제외')
print()
print('🎉 관계 유형을 나누니 엔진이 정확한 판단을 합니다!')

### 🔧 직접 해보기 #1: 새로운 관계 추가

아래 코드에서 주석(`#`)을 해제하고 실행해 보세요!

> 💡 **미션**: `C창고` 노드를 만들고, `B제품`과 `STORES` 관계를 연결한 뒤, 전체 관계를 출력해 보세요.

In [ ]:
# ✏️ 아래 주석을 해제하고 실행해 보세요!

# dozerdb_graph.add_node('C창고', type='Warehouse', definition='경기도 물류센터')
# dozerdb_graph.add_edge('C창고', 'B제품', key='STORES', quantity=1000)

# 전체 관계 확인
print('=== 현재 전체 관계 ===')
for u, v, key, data in dozerdb_graph.edges(keys=True, data=True):
    print(f'  {u} —[{key}]→ {v}  {data}')

---
## 4. 엣지 속성이 왜 중요한가 — 신뢰도, 유효 기간

> 🎯 **이 섹션에서 확인할 것**
> - DataHub Glossary는 관계에 속성을 붙일 수 없다
> - 그래프 DB에서는 관계 자체에 신뢰도, 유효 기간 등을 기록할 수 있다
> - 조직 개편 같은 실무 변화를 시간축으로 추적할 수 있다

> DataHub Glossary에서 "A RelatedTo B"를 설정하면, 그 관계에 아무것도 더 붙일 수 없다.

실무에서 가장 흔한 **조직 개편** 사례를 시뮬레이션합니다.

In [ ]:
# 실무 시나리오: 조직 개편으로 부서-제품 매핑이 변경된 경우
G_temporal = nx.MultiDiGraph()

G_temporal.add_node('영업1팀', type='Department')
G_temporal.add_node('영업2팀', type='Department')
G_temporal.add_node('프리미엄라인', type='ProductLine')

# 과거 관계 (2024년까지)
G_temporal.add_edge('영업1팀', '프리미엄라인', key='MANAGES',
    valid_from='2023-01-01', valid_to='2024-12-31', confidence=0.95, source='HR시스템')

# 현재 관계 (2025년~)
G_temporal.add_edge('영업2팀', '프리미엄라인', key='MANAGES',
    valid_from='2025-01-01', valid_to=None, confidence=0.95, source='HR시스템')

# 시간축 조회
query_date = '2026-02-18'
print(f'=== 시간축이 있는 관계 ===')
print(f'📅 조회 시점: {query_date}')
print()

for u, v, key, data in G_temporal.edges(keys=True, data=True):
    valid_to = data.get('valid_to')
    is_current = valid_to is None or valid_to >= query_date
    status = '✅ 현재 유효' if is_current else '⛔ 만료됨'
    print(f'  {u} —[{key}]→ {v}  [{status}]')
    print(f'      유효기간: {data["valid_from"]} ~ {valid_to or "현재"}')
    print(f'      confidence: {data["confidence"]}, source: {data["source"]}')
    print()

current = [u for u, v, key, data in G_temporal.edges(keys=True, data=True)
           if key == 'MANAGES' and (data.get('valid_to') is None or data['valid_to'] >= query_date)]
print(f'→ 현재 프리미엄라인 담당: {current[0]}')
print(f'→ DataHub Glossary에서는 이런 시간축 관리가 불가능!')

In [ ]:
# 📊 시각화: 시간축 관계 (과거 vs 현재)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
pos = {'영업1팀': (0, 1), '영업2팀': (0, -1), '프리미엄라인': (2, 0)}
nc = ['#e74c3c', '#3498db', '#f39c12']

for ax, title, active, inactive in [
    (ax1, '📅 2024년: 영업1팀 담당', ('영업1팀', '프리미엄라인'), ('영업2팀', '프리미엄라인')),
    (ax2, '📅 2025년~현재: 영업2팀 담당', ('영업2팀', '프리미엄라인'), ('영업1팀', '프리미엄라인'))]:
    nx.draw_networkx_nodes(G_temporal, pos, ax=ax, node_color=nc, node_size=2500, alpha=0.9)
    nx.draw_networkx_labels(G_temporal, pos, ax=ax, font_family=FONT_NAME, font_size=11, font_color='white', font_weight='bold')
    nx.draw_networkx_edges(G_temporal, pos, ax=ax, edgelist=[active], edge_color='#27ae60', arrows=True, arrowsize=20, width=3)
    nx.draw_networkx_edges(G_temporal, pos, ax=ax, edgelist=[inactive], edge_color='#bdc3c7', arrows=True, arrowsize=20, width=1, style='dashed')
    ax.set_title(title, fontsize=13, fontfamily=FONT_NAME)
    ax.axis('off')

plt.suptitle('시간축 관계: 조직 개편에 따른 담당 변경', fontsize=14, fontfamily=FONT_NAME, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 🔧 직접 해보기 #2: 조회 시점 바꿔보기

아래 코드에서 `query_date`를 2024년 날짜로 바꿔보세요. 결과가 어떻게 달라지나요?

In [ ]:
# ✏️ 날짜를 바꿔보세요! (예: '2024-06-15')
query_date = '2024-06-15'

print(f'📅 조회 시점: {query_date}')
print()

for u, v, key, data in G_temporal.edges(keys=True, data=True):
    valid_to = data.get('valid_to')
    is_current = (data.get('valid_from', '') <= query_date and
                  (valid_to is None or valid_to >= query_date))
    status = '✅ 유효' if is_current else '⛔ 만료/미시작'
    print(f'  {u} —[{key}]→ {v}  [{status}]')

current = [u for u, v, key, data in G_temporal.edges(keys=True, data=True)
           if key == 'MANAGES' and data.get('valid_from', '') <= query_date
           and (data.get('valid_to') is None or data['valid_to'] >= query_date)]
print()
if current:
    print(f'→ {query_date} 시점 담당: {current[0]}')
else:
    print('→ 해당 시점에 유효한 담당 부서가 없습니다.')

---
## 5. 파생 지표를 그래프로 표현: `CALCULATED_FROM`

> 🎯 **이 섹션에서 확인할 것**
> - 계산식(순매출 = 총매출 - 반품 - 에누리)을 그래프 관계로 표현
> - 계산식 변경 시 관계만 수정하면 자동 반영
> - Excel에 묻힌 계산식보다 추적이 쉽다

> "순매출 = 총매출 - 반품 - 에누리"를 Excel이 아닌 그래프 관계로 관리

In [ ]:
# 파생 지표 그래프 구축
metrics = nx.MultiDiGraph()

terms = {
    '순매출': {'type': 'DerivedMetric', 'definition': '총매출에서 반품과 에누리를 뺀 금액'},
    '총매출': {'type': 'BaseMetric',    'definition': '할인 전 전체 매출액'},
    '반품':   {'type': 'BaseMetric',    'definition': '고객 반품에 의한 차감액'},
    '에누리': {'type': 'BaseMetric',    'definition': '할인 및 에누리 차감액'},
}
for name, attrs in terms.items():
    metrics.add_node(name, **attrs)

metrics.add_edge('순매출', '총매출', key='CALCULATED_FROM', operator='base',     order=1)
metrics.add_edge('순매출', '반품',   key='CALCULATED_FROM', operator='subtract', order=2)
metrics.add_edge('순매출', '에누리', key='CALCULATED_FROM', operator='subtract', order=3)

def extract_formula(graph, metric_name):
    calc_edges = sorted(
        [(v, data) for u, v, key, data in graph.edges(keys=True, data=True)
         if u == metric_name and key == 'CALCULATED_FROM'],
        key=lambda x: x[1]['order'])
    parts = []
    for term_name, data in calc_edges:
        op = data['operator']
        if op == 'base':     parts.append(term_name)
        elif op == 'subtract': parts.append(f'- {term_name}')
        elif op == 'add':      parts.append(f'+ {term_name}')
    return ' '.join(parts)

print(f'📊 순매출 = {extract_formula(metrics, "순매출")}')
print()
print('💡 그래프에서 계산식이 자동 추출됩니다.')

In [ ]:
# 📊 시각화: 파생 지표 계산 그래프
fig, ax = plt.subplots(figsize=(10, 6))
pos = {'순매출': (1, 2), '총매출': (0, 0), '반품': (1, 0), '에누리': (2, 0)}

nc = ['#e74c3c' if metrics.nodes[n]['type'] == 'DerivedMetric' else '#3498db' for n in metrics.nodes()]
nx.draw_networkx_nodes(metrics, pos, ax=ax, node_color=nc, node_size=2500, alpha=0.9)
nx.draw_networkx_labels(metrics, pos, ax=ax, font_family=FONT_NAME, font_size=13, font_color='white', font_weight='bold')

el = {}
for u, v, key, data in metrics.edges(keys=True, data=True):
    el[(u, v)] = 'base' if data['operator'] == 'base' else 'subtract (-)'
nx.draw_networkx_edges(metrics, pos, ax=ax, edge_color='#2c3e50', arrows=True, arrowsize=20, width=2)
nx.draw_networkx_edge_labels(metrics, pos, ax=ax, edge_labels=el, font_family=FONT_NAME, font_size=10, font_color='#e67e22')

ax.set_title(f'파생 지표: 순매출 = {extract_formula(metrics, "순매출")}', fontsize=14, fontfamily=FONT_NAME, fontweight='bold')
ax.text(0.5, -0.05, '🔴 파생 지표    🔵 기본 지표', transform=ax.transAxes, ha='center', fontsize=12, fontfamily=FONT_NAME)
ax.axis('off')
plt.tight_layout()
plt.show()

### 🔧 직접 해보기 #3: 계산식 변경

아래 코드의 주석을 해제하면 "할인" 항목이 추가됩니다. 계산식이 자동으로 반영되는지 확인해 보세요!

In [ ]:
# ✏️ 주석을 해제하고 실행해 보세요!

# metrics.add_node('할인', type='BaseMetric', definition='프로모션 할인 차감액')
# metrics.add_edge('순매출', '할인', key='CALCULATED_FROM', operator='subtract', order=4)

print(f'📊 순매출 = {extract_formula(metrics, "순매출")}')
print()
print('💡 주석을 해제하면 "- 할인"이 자동으로 추가됩니다!')
print('   → 관계 하나만 추가하면 계산식이 자동 반영')
print('   → Excel에서 시트를 찾아 수정할 필요 없음')

---
## 6. 시각화: 전체 온톨로지 그래프

> DataHub UI는 Lineage(데이터 계보) 탐색에 맞춰져 있어, 다대다 그래프를 탐색하는 화면 자체가 없다.

지금까지 만든 모든 노드와 관계를 하나의 그래프로 시각화합니다.

In [ ]:
# 전체 온톨로지 그래프
full = nx.MultiDiGraph()

# 공장-제품
full.add_node('A공장', type='Factory')
full.add_node('B제품', type='Product')
full.add_edge('A공장', 'B제품', key='MANUFACTURES')
full.add_edge('A공장', 'B제품', key='STOCKS')

# 조직-라인
full.add_node('영업2팀', type='Department')
full.add_node('프리미엄라인', type='ProductLine')
full.add_edge('영업2팀', '프리미엄라인', key='MANAGES')

# 파생 지표
full.add_node('순매출', type='DerivedMetric')
full.add_node('총매출', type='BaseMetric')
full.add_node('반품', type='BaseMetric')
full.add_node('에누리', type='BaseMetric')
full.add_edge('순매출', '총매출', key='CALC_FROM')
full.add_edge('순매출', '반품', key='CALC_FROM')
full.add_edge('순매출', '에누리', key='CALC_FROM')

# 시각화
fig, ax = plt.subplots(figsize=(14, 9))
pos = nx.spring_layout(full, seed=42, k=2.5)

type_colors = {
    'Factory': '#e74c3c', 'Product': '#3498db', 'Department': '#9b59b6',
    'ProductLine': '#f39c12', 'DerivedMetric': '#e67e22', 'BaseMetric': '#27ae60'
}
nc = [type_colors.get(full.nodes[n].get('type', ''), '#95a5a6') for n in full.nodes()]

nx.draw_networkx_nodes(full, pos, ax=ax, node_color=nc, node_size=2500, alpha=0.9)
nx.draw_networkx_labels(full, pos, ax=ax, font_family=FONT_NAME, font_size=11, font_color='white', font_weight='bold')

el = {(u, v): key for u, v, key in full.edges(keys=True)}
nx.draw_networkx_edges(full, pos, ax=ax, edge_color='#2c3e50', arrows=True, arrowsize=15, width=1.5,
                       connectionstyle='arc3,rad=0.1')
nx.draw_networkx_edge_labels(full, pos, ax=ax, edge_labels=el, font_family=FONT_NAME, font_size=9, font_color='#7f8c8d')

# 범례
legend_items = [(c, t) for t, c in type_colors.items() if t in [full.nodes[n].get('type', '') for n in full.nodes()]]
for color, label in legend_items:
    ax.scatter([], [], c=color, s=100, label=label)
ax.legend(loc='upper left', fontsize=10, framealpha=0.9)

ax.set_title('전체 온톨로지 그래프 — 다대다 그물망 구조', fontsize=16, fontfamily=FONT_NAME, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.show()

print('💡 DataHub에는 이런 그래프를 탐색하는 화면이 없습니다.')
print('   그래프 DB + 시각화 도구 조합이 필요한 이유입니다.')

---
## 7. 정리: 되는 것 vs 안 되는 것

이 실습에서 직접 확인한 내용을 정리합니다.

| | DataHub Glossary | DozerDB (그래프 DB) |
|---|---|---|
| **용어 정의** | ✅ 가능 (name, definition) | ✅ 가능 |
| **기본 관계** | ✅ 4종 (IsA, HasA, RelatedTo, Values) | ✅ 무제한 |
| **세분화된 관계** | ❌ 불가 (MANUFACTURES, STOCKS 등) | ✅ 자유롭게 정의 |
| **관계에 속성** | ❌ 불가 | ✅ 신뢰도, 유효기간 등 |
| **시간축 관리** | ❌ 불가 | ✅ valid_from, valid_to |
| **파생 지표 관리** | ❌ 불가 | ✅ CALCULATED_FROM |
| **그래프 시각화** | ❌ Lineage만 가능 | ✅ 다대다 탐색 가능 |
| **관계 확장** | ❌ 재배포 필요 | ✅ 즉시 추가 |

### 결론

> **DataHub Glossary**는 **용어 사전**으로서는 훌륭하지만, **온톨로지 저장소**로는 표현력이 모자라다.
>
> → 역할을 나누자: **DataHub**(용어 정의 원천) + **DozerDB**(세분화된 관계, 엣지 속성, 그래프 추론)

다음 글에서는 SKOS 호환 레이어를 왜 넣었는지 다룹니다.

---

*DataNexus를 설계하고 구축하는 과정을 기록합니다. [GitHub](https://github.com/biz-agentic-ai) | [LinkedIn](https://www.linkedin.com/in/leejuno/)*